In [9]:
import tifffile
import scipy.io
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import filters,measure
from skimage.filters import threshold_local

If you don't have a mask image, we can try to generate the pinhole position with original iamge by stacking them together. Compared to the real mask, this method will have a smalller ROI.

In [10]:
# Here please specify the path to the folder containing the tiff files. Ideally, they should be the mask files. 
# but you can use the images which you are going to analysis.
path = r'/Volumes/Tianxiao/Spinning_Disk/Mask/'
frames = 500

In [11]:
def generate_df(root,frame):
    image = []
    for i in range(1,frame+1):
        

        mat = root+'Frame_'+str(i)+'/worksapce.mat'
        # Load the .mat file
        mat_data = scipy.io.loadmat(mat)
        
        
        # Access a specific variable by name
        variable_name = 'pixxx'  # Replace with the actual variable name
        if variable_name in mat_data:
            # image = np.mat_data[variable_name]
            specific_data = mat_data[variable_name]
        # if i ==1:
        #     image = specific_data
        # else:
        #     image = np.stack([image,specific_data],axis = 0)
            image.append(specific_data)
    Image = np.array(image)
    
    return Image

In [12]:
def stack_image(image):
    mean_image = np.mean(image, axis = 0)
    return mean_image

In [13]:
def threshold(image,value):
    threshold_value=filters.threshold_otsu(image) 
    #threshold_value = value
    print(f'Threshold value is {threshold_value}')
    binary_image = image > threshold_value
    return binary_image

In [14]:
def save_image(image,Type):
    if Type == 0:
        tifffile.imwrite(path+'origin.tif', image)
    elif Type == 1:
        imsr = Image.fromarray(image)
        imsr.save(path+'binary_mask.tif')
    elif Type == 2:
        tifffile.imwrite(path+'masked.tif',image)

In [15]:
origin_image = generate_df(path,frames)
stacked_image = stack_image(origin_image)
binary_image = threshold(stacked_image,5000)

Threshold value is 1148.35505859375


In [16]:
save_image(binary_image,1)
save_image(origin_image,0)

So far, you will get the binary image (ROI) and also the timelapse of all the frames you had taken in one tiff file. You are now ready to move to the second code.